Let's Build a Routing Engine

Good map-based routing is dependent on an accurate, well-defined, and updated graph network. Graph algorithms are treated as an advanced topic in computer science since few of the graph problems such as the traveling salesman problem (TSP) are considered NP-complete problems. But who says we have to flirt with the NP-completeness paradigm to build a simple routing engine? With open source road traffic data and public transit feeds, routing engines will no longer be a black box.

The following topics will be covered in this chapter:

- Fundamentals of graph data structure
- Shortest path analysis on a simple graph
- Building a graph based on a road network
- Shortest path analysis on the road network graph

**Fundamentals of graph data structure**

Graphs can be effectively used to model and solve routing problems through road and public transit networks. Graphs can be designed to model and predict financial transactions and even complex social networks (yeah, blame a graph algorithm the next time Facebook or LinkedIn makes an unfamiliar or unsolicited friend suggestion or professional connection). Despite its versatility, the graph universe is made up of just two simple, easily relatable components, namely, nodes and edges. In a road network, a node might represent a road intersection and an edge might very well represent the road segment itself. The convention is that an edge is an entity that always connects two nodes

Simple edge

Let's fire up a new Google Colab Notebook and build our first graph using a Python library known as networkx. By default, the networkx library is installed in the Google Colab environment. If not, be sure to install it using pip or conda or a similar package manager. The following command should work in most Jupyter Notebook coding environments

In [ ]:
!pip install networkx

The networkx library provides a clean and efficient data structure to define and work with graphs. The simplicity of the networkx library is the most appealing factor for adopting this library for this chapter. Let's get started with networkx and create our first graph with just four lines of code

In [ ]:
import networkx as nx
G = nx.Graph()
G.add_edge('A','B')
G.add_edge('B','C')

An elementary graph with nodes and edges

Just like that, we have defined a graph object, G, and we have added two edges to it. The first edge connects the 'A' and 'B' nodes, and the second edge connects nodes 'B' and 'C'. 

**Directional graphs**

These edges in the preceding graph didn't have a direction. But edges can have a direction. If you think of the road network analogy, there are one-way roads, in which you can only drive along in one direction, but most roads are bidirectional


One way roads

This can be modeled in networkx by instantiating a directional graph (digraph). In a digraph, the position of the nodes in the edge definition determines the direction of the edge. The convention is the first node in the edge definition is the from node or source node and the second node is the to node or target node or destination node. Let's instantiate a directional graph with the following code snippet

In [ ]:
H = nx.DiGraph()
H.add_edge('B', 'A')
H.add_edge('B','C')

Plotting the digraph, H, yields the following plot. Notice the arrows in the diagram indicating the direction of the edge

A directed graph

The preceding lines of code mean that there are three nodes, 'A', 'B', and 'C', and there's a connection between B and A and not the other way around. It also means that there's a connection between B and C, but not between C to B. Digraphs are very important in modeling real-world networks, especially road networks.

If a road segment is bidirectional, you might have to add two different edges between the same nodes,  as follows

In [ ]:
I = nx.DiGraph()
I.add_edge("A", "B")
I.add_edge("B", "A")
I.add_edge("A", "C")

In the following plot, notice the arrows. Edge AB has two arrows pointing in opposite directions

**Weighted graphs**

In the previous examples, each edge is considered to have a unit weight. What that means is that the cost of traveling from one node to another through an edge is the same. This need not always be the case. In the case of a road network, each road segment is different from each other in terms of length and time (taken to traverse it). So, if we are going to represent these road segments as edges, we need to make sure that the edges have different costs or weights.

The networkx library allows us to add weight to an edge, which is demonstrated in the code, as follows

In [ ]:
import networkx as nx

#Create a weighted graph
G=nx.DiGraph()
G.add_edge('A','B',weight=6)
G.add_edge('A','C',weight=2)
G.add_edge('C','D',weight=4.5)
G.add_edge('C','E',weight=5)
G.add_edge('C','F',weight=6)
G.add_edge('A','D',weight=3)

A weighted graph with the width of edges representing the weight of the edge

In this section, we had a good introduction to the components and types of graph data structures. In the next section, we will look into the popular analyses commonly performed using graphs. 

**Shortest path analysis on a simple graph**

Suppose you want to connect to Barack Obama through LinkedIn; how many degrees of connection do you have to go through to reach Obama? A first-degree connection is someone who is connected to you on LinkedIn. A second-degree connection is someone who is connected to your first-degree connection and so on. Assuming that each of your connections and their connections respectively are interested and ready to help you network with the Obama, research says that it only takes an average of 5 degrees of connection for you to connect with Obama, or anyone in the world, for that matter. In other words, the shortest path between any of us and Obama is less than or equal to five. That sounds strange, right? This is known as the small-world phenomenon. And, fortunately, the shortest path between This author and Obama is only three.

This means that someone This author knows on LinkedIn knows someone else who is connected to Obama


Now, This author didn't know that Obama is a third-degree connection until he looked up this information on LinkedIn before writing this. But how did LinkedIn compute this degree of separation? LinkedIn knew this fact because it is continuously running the shortest path algorithm for each member on its network and is able to calculate almost in realtime that the shortest path between This author and Obama is three (connections).

Let's look at some code to understand this. Let's create a hypothetical network of friends, centered around a person called JK and link any two individuals in the network to each other with an edge connection if they are friends

In [ ]:
L = nx.Graph()
L.add_edge("JK", "Ashish")
L.add_edge("JK", "Athulya")
L.add_edge("JK", "Peter")
L.add_edge("JK", "Eric")
L.add_edge("Peter", "Derrick")
L.add_edge("Peter", "Vijay")
L.add_edge("Eric", "Enoch")
L.add_edge("Athulya", "Jeremiah")
L.add_edge("Derrick", "Obama" )
L.add_edge("Jeremiah", "Aaron")
L.add_edge("Aaron", "Obama")

JK's friend network

Now, let's see who can connect JK to Obama. Well, Athulya can connect JK to Obama through Jeremiah, and Jeremiah through Aaron. But voila, Peter can connect JK to Obama through his friend Derrick. Hence, the shortest path for JK to Obama is through Peter and Derrick, respectively, and the shortest path length for JK to Obama is three. This can easily be found using the shortest_path_lenth() method in networkx

In [ ]:
nx.shortest_path_length(L, 'JK')

The preceding dictionary returned by the shortest_path_length() method lets us know the shortest path length for JK to everyone else in the network. Of course, the shortest path length to JK himself is 0.

Now, the preceding shortest path algorithm is operating on edges with unit weight: that is, there is an assumption that all of the connections are equally likely to connect the source to the target. And LinkedIn probably doesn't weight your connections; it doesn't need to. But other types of networks, such as a road network, isn't unweighted networks. They are, in fact, weighted directional networks.

Shortest path algorithms also work on such graphs and try to optimize for edge weight. In order words, they try to provide a solution for traversing from one node to another target node by accumulating the minimum cost, in terms of edge weight. This sounds very intuitive. An experienced driver in a city doesn't need any navigation app to take the shortest path between any two points: they intuitively do that. But it is highly improbable that the driver is able to factor in current traffic or road accidents and dynamically change their route or drive in the shortest time possible to a location they've never been to. For novice drivers and tourists, they are absolutely dependent on a navigation app to solve their shortest route problem. So, let's solve it for them. There are a lot of shortest paths algorithms, but let's review the basics of one of the most popular shortest path algorithm before jumping into the code

**Dijkstra's algorithm**

If you find Dijkstra to be a tongue twister, you are like everyone else in the world minus the Dutch. But this guy with a tongue-twister name is credited with introducing one of the most efficient algorithms for solving routing problems, thereby giving the algorithm his name as well as tongue-sprain (if such a thing exists) for generations of computer scientists to come. Let's understand how Dijkstra's algorithm works using a simple network represented as follows

Let's represent this graph as a list of tuples, as shown in the following code. The first two elements in each tuple represent the nodes being connected and the third value represents the weight of the edge between the corresponding nodes

In [ ]:
eList = [(1, 2, 7.0), (1, 4, 5.0),
(2, 3, 8.0), (2, 4, 9.0), (2, 5, 7.0),
 (3, 5, 5.0), (4, 5, 15.0), (4, 6, 6.0),
 (5, 6, 8.0), (5, 7, 9.0), (6, 7, 11.0)]

The networkx library can ingest this list of tuples and construct a graph from it using the add_weighted_edges_from() method

In [ ]:
R = nx.Graph()
R.add_weighted_edges_from(eList)

**Calculating Dijkstra's shortest path**

Let's say our goal is to reach node 7 from node 1 by accumulating the minimum cost. Let's ask the graph right away what's the shortest path from node 1 to 7

In [ ]:
nx.dijkstra_path(R, 1, 7)

This means that the shortest path to reach node 7 from node 1 is through nodes 4 and 6 respectively. The traversal one has to make is as follows: edge 1-4, then edge 4-6 and, finally, edge 6-7.

**Calculating Dijkstra shortest path length**

If the graph were an unweighted graph, the shortest path length would have been three, since only three hops were needed to reach node 7 from node 1. Since it is a weighted graph, to calculate the accumulated cost, we need to add up the weights of the edges 1–4, 4–6, and 6–7, which is 5.0, 6.0, and 11.0 respectively. Hence, the shortest path length is 22.0. Let's confirm this with the dijkstra_path_length() method, as follows

In [ ]:
nx.dijkstra_path_length(R, 1, 7)

The arguments for the preceding function are the graph object, the source node, and the target node respectively. When in doubt, always enter the method name prefixed by the ? sign in a Colab (or Jupyter) Notebook cell

In [ ]:
?nx.dijkstra_path_length()

In docstring, you might have noticed that there's an optional argument, weight. Since the default value is weight itself, we didn't have to mention this explicitly.

If we need to verify the data attributes present in the edges, we can just reckon the edge data from the graph object, as follows

In [ ]:
R.edges(1, data=True)

The following response confirms that we have data for our edges with the weightattribute name, at least for edges incident to node 1

In [ ]:
EdgeDataView([(1, 2, {'weight': 7.0}), (1, 4, {'weight': 5.0})])

**Calculating single source Dijkstra path length**

Let's try something even more interesting. Let's try to calculate the Dijkstra's shortest path length to each and every node from the source node, that is, node 1. We can accomplish this using the following method: nx.single_source_dijkstra_path_length()

In [ ]:
nx.single_source_dijkstra_path_length(G, source, cutoff=None, weight='weight')

Of course, feel free to look up the entire docstring by prepending the method name with a single question (?), or if you want to venture into the source code of the method, prepend a double-question mark (??).

The method signature suggests that the graph object and the source node are required parameters and the two other parameters are optional. By now, we know what the weight parameter is. The cutoff parameter accepts a cutoff value for the accumulated weights. We'll try to understand this once we execute the method without the optional parameters first

In [ ]:
nx.single_source_dijkstra_path_length(R, 1)

In the response, the keys of the dictionary represent the node IDs and the values of the dictionary items represent the shortest path length to the corresponding node from the source node. From our earlier queries, we know very well that the shortest distance length to node 7 from node 1 is 22.0 and the distance to node 1 itself is 0, which can be observed in the preceding response. 

Let's try to get the shortest path lengths to the entire network from a different source node, say, node 5, and look at the response:

In [ ]:
nx.single_source_dijkstra_path_length(R, 5) 

The following response provides us with some naive facts, such as the fact that the shortest path distance to itself is 0, and some interesting insights such as the furthest nodes from node 5 are node 1 and node 4; the cost to reach both of these nodes is 14.0

Let's assume that that the accumulated weight values in the preceding response refers to the distance, in miles, it takes to travel from location 5 to all possible neighboring locations. Say we are evaluating all possible locations neighboring location 5, but we have a hard limit of 10 miles that a location in the network can be away from location 5. How will the preceding method call change, when we need to introduce such a criterion? It's here that the cutoff parameter comes into play. Plug in the value of 10.0 to the cutoff parameter in the preceding code. The scenario is illustrated as follows

The code implementation for the scenario translates as shown as follows

In [ ]:
nx.single_source_dijkstra_path_length(R, 5, cutoff=10)

As you would expect, node 1 and node 4 are excluded in the preceding response. This is quite useful while querying large graphs and we will be using this method to create something known as an isochrone a little later

**Turning a simple DataFrame into graphs**

We dealt with constructing a graph by defining the individual edges and an edge list manually. This can be cumbersome for large graphs. But we can import these from other sources, such as edge list files and noticeably from a pandas DataFrame. The DataFrame just needs to have a source column, a target column, and an optional weight column.

Let's start with a simple pandas DataFrame and then deal with larger data frames very soon:

In [ ]:
import pandas as pd
edges = pd.DataFrame({ 'source': [0, 1, 2],
                       'target': [2, 2, 3],
                       'dist': [3, 4, 5],
                       'time' :[10, 12, 15]
                     })
edges

When it is displayed, the pandas DataFrame looks like this. The source and target columns probably represent the edges and the directionality of the edges. So, this should definitely be converted into a digraph (nx.DiGraph()). There are two other attribute fields, dist and time, both of which could potentially act as a weight of the edges


A simple pandas DataFrame that could be converted to a graph

A method called nx.from_pandas_edgelist() can be used to convert a pandas DataFrame into a graph



In [ ]:
G = nx.from_pandas_edgelist(edges, edge_attr=True, create_using= nx.DiGraph())

In the preceding code, we are passing edges to the from_pandas_edgelist() method and setting the edge_attr parameter to be True. We are also indicating that a digraph should be created from the DataFrame through the create_using parameter. That's it: we have converted a DataFrame into a networkx graph, upon which we can perform all kinds of graph operations. 

 

The following lines of code can be used to visualize a graph:

In [ ]:
pos=nx.spring_layout(G)
nx.draw_networkx(G, pos, edge_color='b', )
edge_labels = nx.get_edge_attributes(G,'time')
nx.draw_networkx_edge_labels(G, pos, font_size=12, edge_labels=edge_labels) 
plt.axis('off')

Visualizing the previous graph by considering time as the weight of the graph

We have gained a foundational knowledge about graph data structure, its representation using the networkx library, and about a few popular analyses performed using graph data structure. In the next section, we will deal with leveraging this data structure to perform analyses on spatial data.